# Advent of Code 2019
## [Day 5: Sunny with a Chance of Asteroids](https://adventofcode.com/2019/day/5)

#### Load Data

In [1]:
import numpy as np
from collections import deque

In [2]:
with open("input.txt") as f:
    input_program = np.array(f.read().split(","), dtype=int)
input_program[:5]

array([  3, 225,   1, 225,   6])

In [3]:
test_program = [3,0,4,0,99]

### Part 1

An Operation is an abstract form of something like add, mul.

An Instruction is a concrete form, with parameters and parameter modes.

A Machine is a structure of (memory, pc, halted, in, out).

So we could write `Machine(program, input).instruction_at(0).call()`

In [4]:
class Operation(object):
    """An Operation is an abstract form of something the machine can do."""
    
    code = None
    num_params = 0
    
    def __init__(self):
        pass

    def __str__(self):
        return self.__class__.__name__.ljust(10)
    
    def operate(self, state, **params):
        raise NotImplementedError("Subclasses should override this")
    
    @classmethod
    def width(cls):
        return 1 + cls.num_params
    
    @classmethod
    def from_code(cls, code):
        operations = {}
        for item in cls.__subclasses__():
            if type(item) == type and issubclass(item, cls):
                operations[item.code] = item
        if code in operations:
            return operations[code]

class Add(Operation):
    code = 1
    num_params = 3

    def operate(self, state, lhs, rhs, result):
        state[result] = state[lhs] + state[rhs]

class Multiply(Operation):
    code = 2
    num_params = 3

    def operate(self, state, lhs, rhs, result):
        state[result] = state[lhs] * state[rhs]

class Input(Operation):
    code = 3
    num_params = 1

    def operate(self, state, result):
        state[result] = state.input.popleft()

class Output(Operation):
    code = 4
    num_params = 1

    def operate(self, state, param):
        state.output.append(state[param])

class JumpIfTrue(Operation):
    code = 5
    num_params = 2

    def operate(self, state, test, addr):
        if state[test] != 0:
            state.pc = state[addr]

class JumpIfFalse(Operation):
    code = 6
    num_params = 2

    def operate(self, state, test, addr):
        if state[test] == 0:
            state.pc = state[addr]
            
class LessThan(Operation):
    code = 7
    num_params = 3

    def operate(self, state, lhs, rhs, result):
        state[result] = int(state[lhs] < state[rhs])

class Equals(Operation):
    code = 8
    num_params = 3

    def operate(self, state, lhs, rhs, result):
        state[result] = int(state[lhs] == state[rhs])
        
class Halt(Operation):
    code = 99
    num_params = 0

    def operate(self, state):
        state.halted = True

In [5]:
class Param(object):
    def __init__(self, mode, value):
        self.mode = mode
        self.value = value
        
    def __str__(self):
        return f"{['*',''][self.mode]}{self.value}"

In [6]:
class Instruction(object):
    """An Instruction is a concrete instance of an Operation and its parameters"""
    
    def __init__(self, mem, pc):
        code = mem[pc]
        opcode, modes = self.decode(code)
        OpClass = Operation.from_code(opcode)
        if OpClass:
            params = mem[pc + 1 : pc + 1 + OpClass.num_params]
        else:
            params = []
        self.code = code
        self.OpClass = OpClass

        self.params = []
        for i in range(len(params)):
            mode = modes[i]
            param = params[i]
            self.params.append(Param(mode, param))

        self.width = 1 + len(params)
            
    @staticmethod
    def decode(instruction):
        opcode = instruction % 100
        modes = (
            (instruction // 100) % 10,
            (instruction // 1000) % 10,
            (instruction // 10000) % 10
        )
        return opcode, modes
    
    def __str__(self):
        param_strs = [str(p) for p in self.params]
        if self.OpClass:
            return ' '.join([self.OpClass.__name__] + param_strs)
        return f"{self.code}"
    
    def __repr__(self):
        return f"[{self}]"

In [7]:
Instruction([1001,2,3,4,5,6,7], 0)

[Add *2 3 *4]

In [8]:
class Intcode(object):
    """An Intcode machine holds the state of a program, its memory and i/o"""
    def __init__(self, program, input=[]):
        self.mem = np.array(program)
        self.halted = False
        self.input = deque(input)
        self.output = []
        self.pc = 0
        
    def instruction_at(self, pc):
        return Instruction(self.mem, pc)
    
    def next_instruction(self):
        return self.instruction_at(self.pc)
    
    def all_instructions(self):
        instrs = []
        pc = 0
        while pc < len(self.mem):
            instr = self.instruction_at(pc)
            instrs.append(instr)
            pc += instr.width
        return instrs
    
    def run_instruction(self, instr):
        instr.OpClass().operate(self, *instr.params)        
    
    def run(self, *input, pc=0, verbose=False):
        if input:
            self.input = deque(input)
        self.pc = pc
        while self.pc < len(self.mem):
            instr = self.next_instruction()
            if verbose:
                print(f"{self.pc: 6d}: {instr}")
            self.pc += instr.width
            self.run_instruction(instr)
            if self.halted:
                break
        return self.output
    
    def __str__(self):
        lines = [
            [f"{pc: 6d} {i}" for pc, i in self.all_instructions()]
        ]
        return '\n'.join(lines)
    
    def __repr__(self):
        return f"<Machine pc:{self.pc}/{len(self.mem)} in:{len(self.input)} out:{self.output}>"
    
    def __getitem__(self, param):
        # Handle dereferencing a param or using an immediate value
        if param.mode == 1:
            return param.value
        elif param.mode == 0:
            return self.mem[param.value]
        
    def __setitem__(self, param, value):
        # Always write to the param as an address
        self.mem[param.value] = value
    
machine = Intcode([1101,2,3,5,99,5,6,7])
machine.all_instructions()

[[Add 2 3 *5], [Halt], [JumpIfTrue *6 *7]]

In [9]:
m = Intcode([1101,2,3,5,99,5,6,7])
m.run(verbose=True)
m.output

     0: Add 2 3 *5
     4: Halt


[]

In [10]:
print(Intcode([1001,2,3,4,5,6,7]).all_instructions())

[[Add *2 3 *4], [JumpIfTrue *6 *7]]


In [11]:
m = Intcode(input_program, input=[1])
m.run()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 14155342]

In [12]:
m.mem[:10]

array([   3,  225,    1,  225,    6,    6, 1101,    1,  238,  225])

In [13]:
input_program[:10]

array([   3,  225,    1,  225,    6,    6, 1100,    1,  238,  225])

In [14]:
m = Intcode(input_program, input=[1])
m.run_instruction(m.next_instruction())

In [15]:
m.mem[225]

1

In [16]:
input_program[225]

0

In [17]:
Intcode([3,9,8,9,10,9,4,9,99,-1,8]).run(8)

[1]

In [18]:
Intcode([3,3,1108,-1,8,3,4,3,99]).run(7)

[0]

In [19]:
Intcode([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9]).run(2)

[1]

In [23]:
Intcode([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]).all_instructions()

[[Input *21],
 [Equals *21 8 *20],
 [JumpIfTrue *20 22],
 [LessThan 8 *21 *20],
 [JumpIfFalse *20 31],
 [JumpIfFalse 0 36],
 [98],
 [0],
 [0],
 [Multiply *21 125 *20],
 [Output *20],
 [JumpIfTrue 1 46],
 [Output 999],
 [JumpIfTrue 1 46],
 [Add 1000 1 *20],
 [Output *20],
 [JumpIfTrue 1 46],
 [98],
 [Halt]]

In [25]:
Intcode(input_program).run(5)

[8684145]